In [2]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping



In [18]:
# map_path = path of 'map.pkl'
map_path = 'data/map.pkl'#sys.argv[1]

# base_path = directory of all the data (train_label.csv, X_train.npy)
base_path = 'feature/fbank4/'#sys.argv[2]


with open(map_path, 'rb') as f:
    map_dict = pickle.load(f)

verified = []
unverified = []

train_label_path = os.path.join(base_path, 'train_label.csv')
Y_train = pd.read_csv(train_label_path)

for i in range(len(Y_train)):
    if Y_train['manually_verified'][i] == 1:
        verified.append(i)
    else:
        unverified.append(i)

Y_un = Y_train.loc[unverified,:]

fname_all = Y_un['fname']
fname_all = np.array(fname_all)

Y_dict = Y_un['label'].map(map_dict)
Y_dict = np.array(Y_dict)

Y_all = []
for i in Y_dict:
    Y_all.append(to_categorical(i, num_classes=41))
Y_all = np.array(Y_all)

filename = os.path.join(base_path, 'fname_unverified.npy')
np.save(filename, fname_all)

filename = os.path.join(base_path, 'y_unverified.npy')
np.save(filename, Y_all)

X_train_path = os.path.join(base_path, 'X_train.npy')
X_all = np.load(X_train_path)

X_un = X_all[unverified, :]
filename = os.path.join(base_path, 'X_unverified.npy')
np.save(filename, X_un)

X_ver = X_all[verified, :]
train_label_path = os.path.join(base_path, 'train_label.csv')
Y_ver = pd.read_csv(train_label_path)
Y_ver = Y_ver.loc[verified,:]
'''
exit()
X_all = X_all[verified, :]
idx = list(range(X_all.shape[0]))
random.shuffle(idx)
xSize = math.ceil(X_all.shape[0] / num_fold)
split_X_path = os.path.join(base_path, 'X')
split_y_path = os.path.join(base_path, 'y')
if not os.path.exists(split_X_path):
    os.makedirs(split_X_path)
if not os.path.exists(split_y_path):
    os.makedirs(split_y_path)
for i in range(num_fold):
    X = X_all[idx[i*xSize:i*xSize+xSize]]
    y = Y_all[idx[i*xSize:i*xSize+xSize]]
    print('Saving fold {}'.format(i+1))
    print('X.shape:', X.shape)
    print('y.shape:', y.shape)
    filename = os.path.join(split_X_path, 'X' + str(i+1) + '.npy')
    np.save(filename, X)
    filename = os.path.join(split_y_path, 'y' + str(i+1) + '.npy')
    np.save(filename, y)
    time.sleep(1)
'''

"\nexit()\nX_all = X_all[verified, :]\nidx = list(range(X_all.shape[0]))\nrandom.shuffle(idx)\nxSize = math.ceil(X_all.shape[0] / num_fold)\nsplit_X_path = os.path.join(base_path, 'X')\nsplit_y_path = os.path.join(base_path, 'y')\nif not os.path.exists(split_X_path):\n    os.makedirs(split_X_path)\nif not os.path.exists(split_y_path):\n    os.makedirs(split_y_path)\nfor i in range(num_fold):\n    X = X_all[idx[i*xSize:i*xSize+xSize]]\n    y = Y_all[idx[i*xSize:i*xSize+xSize]]\n    print('Saving fold {}'.format(i+1))\n    print('X.shape:', X.shape)\n    print('y.shape:', y.shape)\n    filename = os.path.join(split_X_path, 'X' + str(i+1) + '.npy')\n    np.save(filename, X)\n    filename = os.path.join(split_y_path, 'y' + str(i+1) + '.npy')\n    np.save(filename, y)\n    time.sleep(1)\n"

In [19]:
Y_ver = Y_ver['label'].map(map_dict)
Y_ver = np.array(Y_ver)
Y_un = Y_un['label'].map(map_dict)
Y_un = np.array(Y_un)
Y_un.shape , Y_ver.shape, X_un.shape , X_ver.shape
X_test = np.load('feature/fbank4/X_test.npy')
X_ver = np.swapaxes(X_ver,1,2)
X_un = np.swapaxes(X_un,1,2)
X_test = np.swapaxes(X_test,1,2)
print(X_ver.shape , X_un.shape, len(Y_ver))

(3710, 88, 64, 1) (5763, 88, 64, 1) 3710


In [132]:
fold = 9

In [133]:
autoencoder = load_model('model/LGD_ConvLSTM_clf_'+str(fold)+'.h5')
autoencoder.summary()
#0 => 0.41240
#1 =>0.41
#2 =>0.51482
#3 =>0.45822
#4 =>0.49596
#5 =>0.50674
#6 =>0.33154
#7 =>0.54717
#8 => 0.37736
#9 => 0.40431


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 88, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 88, 64, 32)   2848        input_8[0][0]                    
__________________________________________________________________________________________________
p_re_lu_8 (PReLU)               (None, 88, 64, 32)   180224      conv2d_57[0][0]                  
__________________________________________________________________________________________________
batch_normalization_92 (BatchNo (None, 88, 64, 32)   128         p_re_lu_8[0][0]                  
__________________________________________________________________________________________________
dropout_85

In [134]:
# 41=>GRU output=>512
# 43 => first layer dense=>256
#45=> first concatensate dense=>768
#47=> second dense layer=>256 #6494 6907
#59=>最後一曾dense layer=>64 #36272 47066
#60=>output layer=>41 #94461 115512
model = Model(inputs = autoencoder.layers[0].input, outputs = autoencoder.layers[60].output)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 88, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 88, 64, 32)   2848        input_8[0][0]                    
__________________________________________________________________________________________________
p_re_lu_8 (PReLU)               (None, 88, 64, 32)   180224      conv2d_57[0][0]                  
__________________________________________________________________________________________________
batch_normalization_92 (BatchNo (None, 88, 64, 32)   128         p_re_lu_8[0][0]                  
__________________________________________________________________________________________________
dropout_85

In [135]:
X_ver_dense = model.predict(X_ver)
X_un_dense = model.predict(X_un)
X_test_dense = model.predict(X_test)
print(X_ver_dense.shape , X_un_dense.shape , X_test_dense.shape)

(3710, 41) (5763, 41) (9400, 41)


In [136]:
#kernel=’rbf’, gamma=20, n_neighbors=7, alpha=0.2, max_iter=30, tol=0.001
#5~9
lp_model = LabelSpreading(kernel='knn',n_jobs=-1,n_neighbors=int(3+6*random.random()),alpha=0.01+0.24*random.random(),
                          max_iter=int(30+21*random.random()),tol = 0.0001+0.0015*random.random())#,n_neighbors=3,alpha=0.05)#, max_iter=50, gamma=10, alpha=0.1,tol=0.0001)
#0~4
# lp_model = LabelSpreading(kernel='rbf',n_jobs=-1, gamma=5+25*random.random(), max_iter=int(30+21*random.random())
#                           , alpha=0.01+0.24*random.random(),tol = 0.0001+0.0015*random.random())
# lp_model = LabelSpreading(kernel='rbf',n_jobs=-1)

lp_model.fit(X_ver_dense,Y_ver)

LabelSpreading(alpha=0.22502332191013757, gamma=20, kernel='knn', max_iter=32,
        n_jobs=-1, n_neighbors=8, tol=0.0011996989248298049)

In [137]:
un_ans = lp_model.predict(X_un_dense)
test_ans = lp_model.predict(X_test_dense)
print(un_ans, test_ans)
print(sum(un_ans),sum(test_ans)) #94461 115512

[ 3 32  4 ... 30 18 18] [32 33  6 ... 39  0  6]
112499 151154


### Save

In [138]:
np.save('feature/fbank4/semi/X_un_dense'+str(fold)+'.npy',X_un_dense)
np.save('feature/fbank4/semi/Y_un_dense'+str(fold)+'.npy',un_ans)
np.save('feature/fbank4/semi/X_test_dense'+str(fold)+'.npy',X_test_dense)
np.save('feature/fbank4/semi/Y_test_dense'+str(fold)+'.npy',test_ans)

In [140]:
np.save('feature/fbank4/semi/X_un.npy',X_un)
np.save('feature/fbank4/semi/X_test.npy',X_test)

## Ens Verified
### step2
* threshold = 0.5 

In [43]:
feature_type='mfcc7'

In [47]:
#encoder ens
df_un_ans = pd.DataFrame()
for fold in range(10):
#     un_ans = np.load('feature/fbank4/semi/Y_un_dense'+str(fold)+'.npy') #unverified data
    un_ans = np.load('feature/fbank4/semi/Y_test_dense'+str(fold)+'.npy') #testing data
    df = pd.DataFrame(un_ans).T
    df_un_ans = df_un_ans.append(df)
col_list = []
for col_num in range(len(df_un_ans.columns)):
    counter = df_un_ans[col_num].value_counts()
    col_list.append(dict(counter))
len(col_list)    
    

9400

In [48]:
take_list=[]
take_label=[]
for i,stats in enumerate(col_list):
    if max(stats.values()) >=10:
        take_list.append(i)
        ens_label = max(stats.keys(), key=(lambda k: stats[k]))
        take_label.append(ens_label)
print(len(take_list))

41


### SAVE



In [46]:
print(np.array(take_label).shape)
# X_un_ver = X_un[take_list]
Y_un_ver = np.array(take_label)
# np.save('feature/mfcc6/semi/fbank4/X_un_ver.npy',X_un_ver)
pickle.dump(take_list,open('feature/'+feature_type+'/semi/fbank4/X_un_ver_list.pkl','wb'))
np.save('feature/'+feature_type+'/semi/fbank4/Y_un_ver.npy',Y_un_ver)

(38,)


In [49]:
print( np.array(take_label).shape)
# X_test_ver = X_test[take_list]
Y_test_ver = np.array(take_label)
# np.save('feature/mfcc6/semi/fbank4/X_test_ver.npy',X_test_ver)
pickle.dump(take_list,open('feature/'+feature_type+'/semi/fbank4/X_test_ver_list.pkl','wb'))
np.save('feature/'+feature_type+'/semi/fbank4/Y_test_ver.npy',Y_test_ver)

(41,)


In [13]:
Y_un_ver.shape

(38,)

***

In [206]:
stats = col_list[1]
print(max(stats.values()))
max(stats.keys(), key=(lambda k: stats[k]))

5


32

In [210]:
X_un[[2,3,4,5]].shape , X_un.shape

((4, 88, 64, 1), (5763, 88, 64, 1))

In [160]:
un_ans = np.load('feature/fbank4/semi/Y_un_dense2.npy')

In [161]:
un_ans.shape
# df = pd.DataFrame(un_ans).T
df2 = pd.DataFrame(un_ans).T
df = pd.merge(df,df2,how='inner',left_index=True,right_index=True)
df.ap

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,5753_y,5754_y,5755_y,5756_y,5757_y,5758_y,5759_y,5760_y,5761_y,5762_y
0,3,32,4,16,32,32,14,9,17,22,...,4,32,22,6,19,6,6,6,1,1


5763

In [178]:
df2 = df2.append(df2)
kk = df2[1].value_counts()

In [181]:
dict(kk).values()

dict_values([16])

In [159]:
df[0].str.apply(lambda x: pd.Series(Counter(x)))

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas